In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'adult_income'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,income
0,0.301370,0.275385,0.412593,0.058115,0.151153,0.117782,0.280489,0.334997,0.02174,0.0,0.397959,0.267856,0
1,0.452055,0.294819,0.412593,0.483271,0.480326,0.492380,0.280489,0.334997,0.00000,0.0,0.122449,0.267856,0
2,0.287671,0.237239,0.185625,0.105751,0.188429,0.117782,0.280489,0.334997,0.00000,0.0,0.397959,0.267856,0
3,0.493151,0.237239,0.067818,0.483271,0.188429,0.492380,0.130886,0.334997,0.00000,0.0,0.397959,0.267856,0
4,0.150685,0.237239,0.412593,0.483271,0.450585,0.470917,0.130886,0.121875,0.00000,0.0,0.397959,0.272946,0


In [5]:
target = 'income'
y = df[target].values
del df[target]
x = df.values

In [6]:
x.shape[1]

12

In [7]:
y

array([0, 0, 0, ..., 0, 0, 1])

In [8]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [9]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [10]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [11]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [12]:
num_epochs = 500

In [13]:
latent_dims = int(x.shape[1]/2)
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [14]:
latent_dims

6

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [16]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 12.964578
Epoch [2 / 500] average loss: 7.175931
Epoch [3 / 500] average loss: 7.106383
Epoch [4 / 500] average loss: 7.086924
Epoch [5 / 500] average loss: 7.078533
Epoch [6 / 500] average loss: 7.070958
Epoch [7 / 500] average loss: 7.069234
Epoch [8 / 500] average loss: 7.071969
Epoch [9 / 500] average loss: 7.065712
Epoch [10 / 500] average loss: 7.063900
Epoch [11 / 500] average loss: 7.062824
Epoch [12 / 500] average loss: 7.061727
Epoch [13 / 500] average loss: 7.060497
Epoch [14 / 500] average loss: 7.057241
Epoch [15 / 500] average loss: 7.056784
Epoch [16 / 500] average loss: 7.055453
Epoch [17 / 500] average loss: 7.055337
Epoch [18 / 500] average loss: 7.055655
Epoch [19 / 500] average loss: 7.054823
Epoch [20 / 500] average loss: 7.053657
Epoch [21 / 500] average loss: 7.055411
Epoch [22 / 500] average loss: 7.052093
Epoch [23 / 500] average loss: 7.052587
Epoch [24 / 500] average loss: 7.052273
Epoch [25 / 500] average loss: 7.050386
Epoc

Epoch [204 / 500] average loss: 7.040810
Epoch [205 / 500] average loss: 7.040956
Epoch [206 / 500] average loss: 7.042060
Epoch [207 / 500] average loss: 7.041055
Epoch [208 / 500] average loss: 7.041880
Epoch [209 / 500] average loss: 7.042454
Epoch [210 / 500] average loss: 7.041697
Epoch [211 / 500] average loss: 7.041424
Epoch [212 / 500] average loss: 7.041524
Epoch [213 / 500] average loss: 7.041280
Epoch [214 / 500] average loss: 7.041055
Epoch [215 / 500] average loss: 7.041632
Epoch [216 / 500] average loss: 7.041786
Epoch [217 / 500] average loss: 7.041387
Epoch [218 / 500] average loss: 7.041896
Epoch [219 / 500] average loss: 7.041365
Epoch [220 / 500] average loss: 7.041677
Epoch [221 / 500] average loss: 7.041854
Epoch [222 / 500] average loss: 7.041918
Epoch [223 / 500] average loss: 7.042234
Epoch [224 / 500] average loss: 7.041554
Epoch [225 / 500] average loss: 7.041158
Epoch [226 / 500] average loss: 7.042231
Epoch [227 / 500] average loss: 7.041324
Epoch [228 / 500

Epoch [404 / 500] average loss: 7.042279
Epoch [405 / 500] average loss: 7.041395
Epoch [406 / 500] average loss: 7.041358
Epoch [407 / 500] average loss: 7.041650
Epoch [408 / 500] average loss: 7.041752
Epoch [409 / 500] average loss: 7.041012
Epoch [410 / 500] average loss: 7.041278
Epoch [411 / 500] average loss: 7.041534
Epoch [412 / 500] average loss: 7.041941
Epoch [413 / 500] average loss: 7.041245
Epoch [414 / 500] average loss: 7.041284
Epoch [415 / 500] average loss: 7.042025
Epoch [416 / 500] average loss: 7.041152
Epoch [417 / 500] average loss: 7.041274
Epoch [418 / 500] average loss: 7.040798
Epoch [419 / 500] average loss: 7.041829
Epoch [420 / 500] average loss: 7.041987
Epoch [421 / 500] average loss: 7.041195
Epoch [422 / 500] average loss: 7.040739
Epoch [423 / 500] average loss: 7.040935
Epoch [424 / 500] average loss: 7.042184
Epoch [425 / 500] average loss: 7.042222
Epoch [426 / 500] average loss: 7.041252
Epoch [427 / 500] average loss: 7.041652
Epoch [428 / 500

In [17]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

5
6


In [18]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [19]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 26.510218
Epoch [2 / 500] average reconstruction error: 15.343235
Epoch [3 / 500] average reconstruction error: 9.024989
Epoch [4 / 500] average reconstruction error: 5.750359
Epoch [5 / 500] average reconstruction error: 3.716187
Epoch [6 / 500] average reconstruction error: 2.410180
Epoch [7 / 500] average reconstruction error: 1.536913
Epoch [8 / 500] average reconstruction error: 1.034564
Epoch [9 / 500] average reconstruction error: 0.724040
Epoch [10 / 500] average reconstruction error: 0.506707
Epoch [11 / 500] average reconstruction error: 0.345988
Epoch [12 / 500] average reconstruction error: 0.241312
Epoch [13 / 500] average reconstruction error: 0.185249
Epoch [14 / 500] average reconstruction error: 0.139141
Epoch [15 / 500] average reconstruction error: 0.114317
Epoch [16 / 500] average reconstruction error: 0.090056
Epoch [17 / 500] average reconstruction error: 0.079695
Epoch [18 / 500] average reconstruction error: 0.06

Epoch [147 / 500] average reconstruction error: 0.000002
Epoch [148 / 500] average reconstruction error: 0.000001
Epoch [149 / 500] average reconstruction error: 0.000002
Epoch [150 / 500] average reconstruction error: 0.000002
Epoch [151 / 500] average reconstruction error: 0.000001
Epoch [152 / 500] average reconstruction error: 0.000001
Epoch [153 / 500] average reconstruction error: 0.000002
Epoch [154 / 500] average reconstruction error: 0.000001
Epoch [155 / 500] average reconstruction error: 0.000001
Epoch [156 / 500] average reconstruction error: 0.000001
Epoch [157 / 500] average reconstruction error: 0.000001
Epoch [158 / 500] average reconstruction error: 0.000002
Epoch [159 / 500] average reconstruction error: 0.000001
Epoch [160 / 500] average reconstruction error: 0.000001
Epoch [161 / 500] average reconstruction error: 0.000000
Epoch [162 / 500] average reconstruction error: -0.000000
Epoch [163 / 500] average reconstruction error: 0.000001
Epoch [164 / 500] average reco

Epoch [289 / 500] average reconstruction error: -0.000003
Epoch [290 / 500] average reconstruction error: -0.000004
Epoch [291 / 500] average reconstruction error: -0.000004
Epoch [292 / 500] average reconstruction error: -0.000004
Epoch [293 / 500] average reconstruction error: -0.000004
Epoch [294 / 500] average reconstruction error: -0.000004
Epoch [295 / 500] average reconstruction error: -0.000004
Epoch [296 / 500] average reconstruction error: -0.000004
Epoch [297 / 500] average reconstruction error: -0.000004
Epoch [298 / 500] average reconstruction error: -0.000004
Epoch [299 / 500] average reconstruction error: -0.000004
Epoch [300 / 500] average reconstruction error: -0.000004
Epoch [301 / 500] average reconstruction error: -0.000004
Epoch [302 / 500] average reconstruction error: -0.000004
Epoch [303 / 500] average reconstruction error: -0.000004
Epoch [304 / 500] average reconstruction error: -0.000004
Epoch [305 / 500] average reconstruction error: -0.000004
Epoch [306 / 5

Epoch [431 / 500] average reconstruction error: -0.000001
Epoch [432 / 500] average reconstruction error: -0.000002
Epoch [433 / 500] average reconstruction error: -0.000003
Epoch [434 / 500] average reconstruction error: -0.000002
Epoch [435 / 500] average reconstruction error: -0.000003
Epoch [436 / 500] average reconstruction error: -0.000004
Epoch [437 / 500] average reconstruction error: -0.000004
Epoch [438 / 500] average reconstruction error: -0.000004
Epoch [439 / 500] average reconstruction error: -0.000003
Epoch [440 / 500] average reconstruction error: -0.000003
Epoch [441 / 500] average reconstruction error: -0.000004
Epoch [442 / 500] average reconstruction error: -0.000003
Epoch [443 / 500] average reconstruction error: -0.000002
Epoch [444 / 500] average reconstruction error: -0.000002
Epoch [445 / 500] average reconstruction error: -0.000003
Epoch [446 / 500] average reconstruction error: 0.000001
Epoch [447 / 500] average reconstruction error: -0.000002
Epoch [448 / 50